In [43]:
import corruptions
import numpy
import torch
import corruption_constants
import numpy as np

In [44]:
c=("gaussian_noise","shot_noise","impulse_noise","motion_blur","zoom_blur","fog","brightness","contrast","jpeg_compression")

In [50]:
corrupt_func = c[0]
severity = 5
# print(torch.min(obs), torch.max(obs), torch.mean(obs), torch.sum(obs), torch.std(obs))
# tensor(0.) tensor(0.0299) tensor(0.0007) tensor(56.0894) tensor(0.0045)

In [691]:
def salt_pepper_noise(x, severity):
    total_pixel = x.numel()
    s = torch.tensor([0.025, 0.05, 0.075, 0.1, 0.125])[severity-1]
    salt_num = (total_pixel * s).int()
    pepper_num = (total_pixel * s).int()
    print(salt_num, pepper_num)
    salt_coords = torch.randint(0,total_pixel,(salt_num,))
    pepper_coords = torch.randint(0,total_pixel,(pepper_num,))
    print(salt_coords, pepper_coords)
    flat_x = x.reshape(total_pixel)
    print(flat_x)
    flat_x[salt_coords] = torch.min(x)
    flat_x[pepper_coords] = torch.max(x)
    print(flat_x)
    x = flat_x.reshape(x.shape)
    return x

rnd = torch.rand(2,1,5,5)
salt_pepper_noise(rnd.clone(),5),rnd,torch.max(rnd)

tensor(6, dtype=torch.int32) tensor(6, dtype=torch.int32)
tensor([36,  7, 39, 24, 48, 23]) tensor([25, 23, 22, 43,  1, 27])
tensor([0.9774, 0.6819, 0.7020, 0.0732, 0.3343, 0.2847, 0.4472, 0.2383, 0.3588,
        0.8818, 0.9452, 0.5245, 0.8466, 0.0476, 0.8830, 0.4512, 0.4478, 0.1876,
        0.4874, 0.4621, 0.7024, 0.6642, 0.6521, 0.5597, 0.9226, 0.9975, 0.4491,
        0.9097, 0.2083, 0.1345, 0.0599, 0.9517, 0.2372, 0.1025, 0.0898, 0.6072,
        0.5609, 0.3400, 0.9464, 0.1130, 0.4365, 0.9986, 0.9200, 0.1846, 0.0863,
        0.6363, 0.2915, 0.9471, 0.1285, 0.0561])
tensor([0.9774, 0.9986, 0.7020, 0.0732, 0.3343, 0.2847, 0.4472, 0.0476, 0.3588,
        0.8818, 0.9452, 0.5245, 0.8466, 0.0476, 0.8830, 0.4512, 0.4478, 0.1876,
        0.4874, 0.4621, 0.7024, 0.6642, 0.9986, 0.9986, 0.0476, 0.9986, 0.4491,
        0.9986, 0.2083, 0.1345, 0.0599, 0.9517, 0.2372, 0.1025, 0.0898, 0.6072,
        0.0476, 0.3400, 0.9464, 0.0476, 0.4365, 0.9986, 0.9200, 0.9986, 0.0863,
        0.6363, 0.2915, 0.9

(tensor([[[[0.9774, 0.9986, 0.7020, 0.0732, 0.3343],
           [0.2847, 0.4472, 0.0476, 0.3588, 0.8818],
           [0.9452, 0.5245, 0.8466, 0.0476, 0.8830],
           [0.4512, 0.4478, 0.1876, 0.4874, 0.4621],
           [0.7024, 0.6642, 0.9986, 0.9986, 0.0476]]],
 
 
         [[[0.9986, 0.4491, 0.9986, 0.2083, 0.1345],
           [0.0599, 0.9517, 0.2372, 0.1025, 0.0898],
           [0.6072, 0.0476, 0.3400, 0.9464, 0.0476],
           [0.4365, 0.9986, 0.9200, 0.9986, 0.0863],
           [0.6363, 0.2915, 0.9471, 0.0476, 0.0561]]]]),
 tensor([[[[0.9774, 0.9986, 0.7020, 0.0732, 0.3343],
           [0.2847, 0.4472, 0.0476, 0.3588, 0.8818],
           [0.9452, 0.5245, 0.8466, 0.0476, 0.8830],
           [0.4512, 0.4478, 0.1876, 0.4874, 0.4621],
           [0.7024, 0.6642, 0.9986, 0.9986, 0.0476]]],
 
 
         [[[0.9986, 0.4491, 0.9986, 0.2083, 0.1345],
           [0.0599, 0.9517, 0.2372, 0.1025, 0.0898],
           [0.6072, 0.0476, 0.3400, 0.9464, 0.0476],
           [0.4365, 0.9986, 0.